# 隐马尔科夫模型Hidden Markov Model HMM

 ## 暴力求解
 1. 生成所有的隐藏状态序列 I
 2. 对每个隐藏状态序列， 计算观测序列出现的概率
 
对于含有 N 个隐藏状态的hmm模型，设生成的序列长度为 T，那么共可生成$N^T$中隐藏状态序列，计算概率需要遍历隐藏序列 T，故时间复杂度为$O(TN^T)$

In [49]:
def generate_hidden_sequences(N,T):
    c = []
    def backtracking(r):
        if len(r) == T:
            c.append([i for i in r])
        else:
            for i in range(N):
                r.append(i)
                backtracking(r)
                r.pop()
    backtracking([])
    return c 
c = generate_hidden_sequences(3,3)
print(len(c))
print(c)

27
[[0, 0, 0], [0, 0, 1], [0, 0, 2], [0, 1, 0], [0, 1, 1], [0, 1, 2], [0, 2, 0], [0, 2, 1], [0, 2, 2], [1, 0, 0], [1, 0, 1], [1, 0, 2], [1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 0], [1, 2, 1], [1, 2, 2], [2, 0, 0], [2, 0, 1], [2, 0, 2], [2, 1, 0], [2, 1, 1], [2, 1, 2], [2, 2, 0], [2, 2, 1], [2, 2, 2]]


In [39]:
from pprint import pprint as pp
class HMM():
    def __init__(self):
        super().__init__()
        self.p = [0.2,0.4,0.4]
        self.A = [[0.5,.2,.3],[.3,.5,.2],[.2,.3,.5]]
        self.B = [[.5,.5],[.4,.6],[.7,.3]]
    def print_hmm(self):
        pp('-----------')
        pp(self.p)
        pp(self.A)
        pp(self.B)
        pp('-----------')
h = HMM()
h.print_hmm()

'-----------'
[0.2, 0.4, 0.4]
[[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
[[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
'-----------'


In [50]:
t = [0,1,0]
print(c)
def prob():
    p = [] 
    for i in c:
        _p = h.p[i[0]] * h.B[i[0]][t[0]]
        for k in range(1,len(i)):
            _p *= h.A[i[k-1]][i[k]] * h.B[i[k]][t[k]]
        p.append(_p)
    return p
p = prob()
print(p)
print(sum(p))
        
        

[[0, 0, 0], [0, 0, 1], [0, 0, 2], [0, 1, 0], [0, 1, 1], [0, 1, 2], [0, 2, 0], [0, 2, 1], [0, 2, 2], [1, 0, 0], [1, 0, 1], [1, 0, 2], [1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 0], [1, 2, 1], [1, 2, 2], [2, 0, 0], [2, 0, 1], [2, 0, 2], [2, 1, 0], [2, 1, 1], [2, 1, 2], [2, 2, 0], [2, 2, 1], [2, 2, 2]]
[0.00625, 0.0020000000000000005, 0.00525, 0.0018, 0.0024000000000000002, 0.0016799999999999999, 0.0009, 0.0010799999999999998, 0.0031499999999999996, 0.006000000000000001, 0.0019200000000000007, 0.005040000000000001, 0.007200000000000001, 0.009600000000000003, 0.00672, 0.0009600000000000001, 0.001152, 0.00336, 0.006999999999999999, 0.0022400000000000002, 0.005879999999999999, 0.007559999999999999, 0.010079999999999999, 0.007055999999999998, 0.0042, 0.005039999999999999, 0.014699999999999998]
0.13021800000000003
